In [23]:
# Imports
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [24]:
# Create connection, database, and collection within Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.atlstation_events_db
collection = db.event_list

In [25]:
# Set up chrome driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/tatianafrattale/.wdm/drivers/chromedriver/mac64/93.0.4577.15/chromedriver] found in cache


In [26]:
# Url set up
url = "https://atlanticstation.com/events/list/"
request = requests.get(url)
browser.visit(url)

# Time sleep that allows for browser to fully load html
time.sleep(10)

In [27]:
# HTML parser
html = browser.html
soup = bs(html, "html.parser")

# Close browser session
browser.quit()

In [28]:
# Make empty lists for data
event_name = []
event_description = []
event_date = []
event_time = []
event_link= []

In [29]:
# Save all event cards to a variable
events = soup.find_all('div', class_='event-card')

In [30]:
# Loop through all events to pull necessary info
for event in events:
    # Find event name
    name = event.find('div', class_='event-card-title').text
    # Find event description
    descrip = event.find('p', class_='hover-card-body').text
    # Find date
    date = event.find('div', class_='event-card-date').text
    # Find time
    time = event.find('div', class_='hover-card-content').text
    # Find link to event
    link = event.find('a', class_='full-link')['href']
    
    # Append data to empty lists
    event_name.append(name)
    event_description.append(descrip)
    event_date.append(date)
    event_time.append(time)
    event_link.append(link)
    
    # Create dictionary
    atl_events= {
        "event_name": name,
        "event_description": descrip,
        "event_date": date,
        "event_time": time,
        "event_link": link
    }
    
    # Insert data into collection in MongoDB
    collection.insert_one(atl_events)
    
    


NameError: name 'result' is not defined